In [10]:
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import numpy as np
import os
from scipy.integrate import ode

## The Double Pendulum
This notebook simulates the trajectory of a double pendulum based on numerically integrating the Hamiltonian equations of motion. This is a system that is fairly easy to formulate analytically but impossible to solve, so it lends itself well to a computational approach.  It also exhibits an interesting quality called chaos in that small changes to the initial conditions of the system result in dramatic changes in the overall trajectory with unpredictable results.

The system's coordinates are related in the following way:
$
x_1 = \frac{L_1}{2}\sin\theta_1,\quad y_1 = -\frac{L_1}{2}\cos\theta_1\\
x_2 = (L_1\sin\theta_1+\frac{L_2}{2}\sin\theta_2),\quad y_2 = -(L_1\cos\theta_1+\frac{L_2}{2}\cos\theta_2)\\
v_1 = \sqrt{\dot{x}_1^2+\dot{y}_1^2},\quad v_2 = \sqrt{\dot{x}_2^2+\dot{y}_2^2}\\
$

Using these coordinates we can write the system's kinetic and potential energy as well as its Hamiltonian:
$
T = \frac{1}{2}(m_1v_1^2+m_2v_2^2) + \frac{1}{2}(I_1\dot{\theta}_1^2+I_2\dot{\theta}_2^2)\\
U = g(m_1y_1+m_2y_2)\\
H = T + U\\
$

The equations of motion governing this system are then given by:
$
\dot{\theta_1} = \frac{\partial H}{\partial p_1} = \\
\dot{\theta_2} = \frac{\partial H}{\partial p_2} = \\
\dot{p_1} = -\frac{\partial H}{\partial \theta_1} = \\
\dot{p_2} = -\frac{\partial H}{\partial \theta_2} = \\
$

This is a set of coupled first order differential equations, which we can solve by numerically integrating.  They depend on a set of constant values ($L_1, L_2, m_1, m_2,$ and $g$) as well as a set of initial conditions ($\theta_{0,1}, \theta_{0,2}, \dot{\theta}_{0,1}, \dot{\theta}_{0,2}$).  The code below solves this system of equations for the users choice of constants and initial conditions and animates the result.

In [30]:
def double_pendulum(t, y, args):
    """ Defines Hamilton's equations for a double_pendulum """
    q1 = y[0] # q1 = theta1, angle
    q2 = y[1] # q2 = theta2, angle
    p1 = y[2] # p1 = omega1, angular velocity
    p2 = y[3] # p2 = omega2, angular velocity
    l1,l2,m1,m2,g = args
    q1_eq = p1
    q2_eq = p2
    p1_eq = -((g*(2*m1+m2)*np.sin(q1) + m2*(g*np.sin(x1-2*x2) + 2*(l2*p2**2 + l1*p1**2*np.cos(q1-q2))*np.sin(q1-q2))) / (2*l1*(m1 + m2 - m2*(np.cos(q1-q2))**2)))
    p2_eq = ((l1*(m1+m2)*p1**2+g*(m1+m2)*np.cos(x1)+l2*m2*p2**2*np.cos(q1-q2))*np.sin(q1-q2))/(l2*(m1+m2-m2*(np.cos(q1-q2))**2))
    return [q1_eq, q2_eq, p1_eq, p2_eq]

def solve_dp(args,time,y0):
    """ Numerically solve the double pendulum equations with scipy """
    t0,t1,dt = time
    r = ode(double_pendulum).set_integrator('dopri5')
    r.set_initial_value(y0, t0).set_f_params(args)
    data=[[t0, y0[0], y0[1], y0[2], y0[3] ]]
    while r.successful() and r.t < t1:
        r.integrate(r.t+dt)
        data.append([r.t, r.y[0], r.y[1], r.y[2], r.y[3] ])
    return np.array(data)

def polar_to_cart(args,angles):
    """ converts q1/q2 from Hamilton's equations into Cartesian xy coordinates for plotting """
    l1,l2,m1,m2,g = args
    time,theta1,theta2 = angles.T
    x1 =  l1*np.sin(theta1)
    y1 = -l1*np.cos(theta1)
    x2 =  l2*np.sin(theta2) + x1
    y2 = -l2*np.cos(theta2) + y1
    return np.array([time,x1,y1,x2,y2]).T

In [32]:
""" Define constants for given run of problem """
""" Change these to try different things! """
theta1_0 = 0.65 # initial displacement angles
theta2_0 = 1.1
p1_0 = 0 # initial momenta
p2_0 = 0
l1 = 0.5 # length of pendula
l2 = 0.5
m1 = 1.0 # mass of the pendula
m2 = 1.0
g  = 9.8 # acceleration due to gravity
args = [l1,l2,m1,m2,g]
fps = 80 # animation fps
total_time = 5 # in seconds
time = [0.0,total_time,1.0/fps] # start, finish, dt
ic   = [np.pi*theta1_0, np.pi*theta2_0, p1_0, p2_0]

In [33]:
d = solve_dp(args,time,ic)
data = from_angle_to_xy(args,d[:,:3])

In [23]:
animate=True
params = {'backend': 'ps',
          'font.size': 20,
          'font.family':'serif',
          'font.serif':['Computer Modern Roman'], # Times, Palatino, New Century Schoolbook, Bookman, Computer Modern Roman
          'ps.usedistiller': 'xpdf',
          'text.usetex': True,
          }
plt.rcParams.update(params)
plt.ion()
fig = plt.figure(figsize=(9.6,5.4),dpi=100) # 1920x1080
fig.subplots_adjust(left=0, right=1, top=1, bottom=0,hspace=0.02,wspace=0.02)
ax = fig.add_subplot(111)
ax.axis('off') # no frame

def plot_trail(data,index):
    """ Plots a line with the trajectory of the tip of pendulum 2 (x2,y2)
    """
    length = 1.0 # seconds
    n = int(length/time[2])
    to_plot = data[:index,:]
    if index < n:
        prepend =  np.tile(data[0],(n-index,1))
        to_plot = np.vstack([prepend,to_plot])
        index = n
    colormap = plt.cm.Greys_r
    colors = [colormap(i) for i in np.linspace(0.0, 1.0, n-1)]
    plots = []
    for j in np.arange(n-1):
        p, = ax.plot(to_plot[index-j-1:index-j+1,3],to_plot[index-j-1:index-j+1,4],
                color=colors[j], zorder=-1)
        plots.append(p)
    return plots

# "plot" returns a tuple of line objects
t,x1,y1,x2,y2 = data[0]
line1, = ax.plot([0.0,x1], [0.0,y1], 'r-')
line2, = ax.plot([x1,x2], [y1,y2], 'r-')
circ1, = ax.plot([x1], [y1], 'ro',markersize=10)
circ2, = ax.plot([x2], [y2], 'ro',markersize=10)
sizeY = 1.2
ax.axis([-sizeY*16/9,sizeY*16/9,-sizeY,sizeY])

frame_names = []
tex=ax.text(0.0,0.85,'',ha="center")

for i,v in enumerate(data):
    t,x1,y1,x2,y2 = v
    # print("t={:.2f}".format(t)) # print statement for debugging, shows timestamp for current frame
    line1.set_data([0.0,x1],[0.0,y1])
    line2.set_data([x1,x2],[y1,y2])
    circ1.set_data([x1],[y1])
    circ2.set_data([x2],[y2])
    
    #plotting the trail significantly slows down plotting, you may want to comment it out
    pls = plot_trail(data,i+1)
    tex.set_text(r"$t={:.3f}$ s".format(t))
    fig.canvas.draw()
    if animate:
        fname = "_tmp{:05d}.png".format(i)
        frame_names.append(fname)
        fig.savefig(fname,bbox_inches='tight')
    for k in pls:
        k.remove()

if animate:
    frames = "_tmp%5d.png"
    frames = "_tmp%5d.png"
    movie_command = "ffmpeg -y -r {:} -i {:} double.mp4".format(fps,frames)
    os.system(movie_command)
    
    for fname in frame_names:
        os.remove(fname)